#

# LP reader

This Notebook develops a .lp reader.

In [1]:
using SparseArrays
using LinearAlgebra
using DataStructures  # For OrderedDict if needed

push!(LOAD_PATH, realpath("../src"))
using lp_problem
using lp_read_LP
using lp_read_mps

<details>
    <summary><h3> LPProblem Struct </h3></summary>

```juila
struct LPProblem
    is_minimize::Bool                     # True if the objective is to minimize
    c::Vector{Float64}                    # Objective function coefficients
    A::SparseMatrixCSC{Float64, Int64}    # Constraint matrix
    b::Vector{Float64}                    # Right-hand side of constraints
    constraint_types::Vector{Char}        # Constraint types ('L', 'G', 'E')
    l::Vector{Float64}                    # Lower bounds for variables
    u::Vector{Float64}                    # Upper bounds for variables
    vars::Vector{String}                  # Variable names
    variable_types::Vector{Symbol}        # Variable types
end
```

</details>

In [ ]:
lp_filepath = "../check/problems/lp_files/1451.lp"
 
lp = read_lp(lp_filepath)

# Accessing different components
println("Objective is to minimize: ", lp.is_minimize)
println("Objective coefficients: ", lp.c)
println("Constraint matrix A: ", lp.A)
println("Right-hand side vector b: ", lp.b)
println("Constraint types: ", lp.constraint_types)
println("Lower bounds: ", lp.l)
println("Upper bounds: ", lp.u)
println("Variables: ", lp.vars)
println("Variable types: ", lp.variable_types)


In [ ]:
println(read_file_to_string(lp_filepath))

In [ ]:
lp_filepath = "../check/problems/lp_files/juLinear_ex1.lp"
 
lp = read_lp(lp_filepath)

# Accessing different components
println("Objective is to minimize: ", lp.is_minimize)
println("Objective coefficients: ", lp.c)
println("Constraint matrix A: ", lp.A)
println("Right-hand side vector b: ", lp.b)
println("Constraint types: ", lp.constraint_types)
println("Lower bounds: ", lp.l)
println("Upper bounds: ", lp.u)
println("Variables: ", lp.vars)
println("Variable types: ", lp.variable_types)

In [ ]:
lp_problem

## Writing LP Files

In [ ]:
lp_filepath = "../check/problems/mps_files/problem.mps"
read_mps_from_file(lp_filepath)

In [4]:
# lp_filepath = "../check/problems/mps_files/test.mps"
# lp = read_mps_from_file(lp_filepath)
# write_lp("../check/problems/lp_files/test.lp", lp)

In [7]:
# lp_filepath = "../check/problems/mps_files/ex_9-7.mps"
# lp = read_mps_from_file(lp_filepath)
# write_lp("../check/problems/lp_files/ex_9-7.lp", lp)

<details>
    <summary> Functions </summary>

```julia
function read_lp_test(filename::String; verbose::Bool=false)
    # Initialize storage variables
    if verbose println("Starting to read LP file: $filename") end
    is_minimize = true
    objective = Dict{String, Float64}()
    constraints = Vector{Dict{String, Float64}}()
    constraint_types = Vector{Char}()
    b = Float64[]
    vars_set = Set{String}()
    variable_types = Dict{String, Symbol}()
    l_bounds = Dict{String, Float64}()
    u_bounds = Dict{String, Float64}()

    # Read all lines from the LP file
    lines = readlines(filename)
    current_section = ""

    for raw_line in lines
        # Clean the line by removing comments and trimming whitespace
        line = strip(split(raw_line, "\\")[1])  # Remove comments starting with '\'
        # Ensure 'line' is a String, not SubString
        line = String(line)
        if line == ""
            if verbose println("Skipping empty line") end
            continue  # Skip empty lines
        end

        # Detect section headers
        lower_line = lowercase(line)
        if verbose println("Processing line: $line") end
        if startswith(lower_line, "minimize")
            current_section = "Objective"
            is_minimize = true
            if verbose println("Detected section: Objective (Minimize)") end
            continue
        elseif startswith(lower_line, "maximize")
            current_section = "Objective"
            is_minimize = false
            if verbose println("Detected section: Objective (Maximize)") end
            continue
        elseif startswith(lower_line, "subject to") || startswith(lower_line, "such that") || startswith(lower_line, "st")
            current_section = "Constraints"
            if verbose println("Detected section: Constraints") end
            continue
        elseif startswith(lower_line, "bounds")
            current_section = "Bounds"
            if verbose println("Detected section: Bounds") end
            continue
        elseif startswith(lower_line, "binary")
            current_section = "Binary"
            if verbose println("Detected section: Binary Variables") end
            continue
        elseif startswith(lower_line, "general") || startswith(lower_line, "integer")
            current_section = "Integer"
            if verbose println("Detected section: Integer Variables") end
            continue
        elseif startswith(lower_line, "end")
            if verbose println("End of LP file detected") end
            break  # End of LP file
        end

        # Parse based on the current section
        if current_section == "Objective"
            if verbose println("Parsing objective line: $line") end

            # Handle the "obj:" prefix, if it exists
            if startswith(lowercase(line), "obj:")
                line = strip(line[5:end])  # Remove "obj:" prefix (5 characters including the space)
            end

            # Remove optional colon (e.g., ":") if present, then proceed with parsing the expression
            expr = startswith(line, ":") ? strip(line[2:end]) : line

            # Parse the expression and update the objective dictionary
            parse_expression(expr, objective, vars_set; verbose=verbose)

        elseif current_section == "Constraints"
            if verbose println("Parsing constraint line: $line") end

            # Example constraint: c1: x1 + x2 <= 10
            m = match(r"^(?:(\w+)\s*:\s*)?(.+?)\s*(<=|>=|=)\s*([+-]?\d+\.?\d*)$", line)
            if m === nothing
                error("Failed to parse constraint: $line")
            end
            _, expr, relation, rhs_str = m.captures
            rhs = parse(Float64, rhs_str)
            push!(b, rhs)
            if verbose println("Parsed constraint RHS: $rhs with relation: $relation") end

            constraint = Dict{String, Float64}()
            # Parse the constraint expression
            constant_term = parse_expression(expr, constraint, vars_set; verbose=verbose)

            # Adjust RHS with the constant term
            b[end] = b[end] - constant_term
            if verbose println("Adjusted RHS: $(b[end]) with constant term: $constant_term") end

            push!(constraints, constraint)
            relation_char = relation == "<=" ? 'L' : (relation == ">=" ? 'G' : 'E')
            push!(constraint_types, relation_char)

        elseif current_section == "Bounds"
            if verbose println("Parsing bounds line: $line") end
            parse_bounds(line, l_bounds, u_bounds, vars_set; verbose=verbose)

        elseif current_section == "Binary"
            if verbose println("Parsing binary variable line: $line") end
            vars = split(line)
            for var in vars
                variable_types[var] = :Binary
                push!(vars_set, var)
                if verbose println("Parsed binary variable $var") end
            end
        elseif current_section == "Integer"
            if verbose println("Parsing integer variable line: $line") end
            vars = split(line)
            for var in vars
                variable_types[var] = :Integer
                push!(vars_set, var)
                if verbose println("Parsed integer variable $var") end
            end
        end
    end

    # Collect and sort variable names
    vars = sort(collect(vars_set))
    var_index = Dict(var => i for (i, var) in enumerate(vars))
    n_vars = length(vars)

    # Objective function coefficients
    c = zeros(Float64, n_vars)
    for (var, coeff) in objective
        if haskey(var_index, var)
            c[var_index[var]] = coeff
            if verbose println("Assigned coefficient for variable $var: $coeff") end
        else
            error("Variable $var in objective not defined in variables.")
        end
    end

    # Constraint matrix A in sparse format
    n_constraints = length(constraints)
    row = Int[]
    col = Int[]
    data = Float64[]

    for (i, constraint) in enumerate(constraints)
        for (var, coeff) in constraint
            if haskey(var_index, var)
                push!(row, i)
                push!(col, var_index[var])
                push!(data, coeff)
                if verbose println("Added constraint for variable $var at row $i: $coeff") end
            else
                error("Variable $var in constraints not defined in variables.")
            end
        end
    end

    A_sparse = sparse(row, col, data, n_constraints, n_vars)

    # Bounds vectors
    l = fill(-Inf, n_vars)
    u = fill(Inf, n_vars)

    for var in vars
        if haskey(l_bounds, var)
            l[var_index[var]] = l_bounds[var]
            if verbose println("Lower bound for $var: $(l_bounds[var])") end
        end
        if haskey(u_bounds, var)
            u[var_index[var]] = u_bounds[var]
            if verbose println("Upper bound for $var: $(u_bounds[var])") end
        end
    end

    # Variable types vector
    variable_types_vec = Symbol[]
    for var in vars
        push!(variable_types_vec, get(variable_types, var, :Continuous))
        if verbose println("Variable type for $var: $(get(variable_types, var, :Continuous))") end
    end

    if verbose println("Successfully parsed LP file") end

    # Create and return the LPProblem struct
    return LPProblem(
        is_minimize,
        c,
        A_sparse,
        b,
        constraint_types,
        l,
        u,
        vars,
        variable_types_vec
    )
end

# Helper function to parse expressions (objective and constraints)
function parse_expression(expr::AbstractString, coeff_dict::Dict{String, Float64}, vars_set::Set{String}; verbose::Bool=false)
    tokens = split(expr, r"(?=[+-])")
    constant_term = 0.0  # Initialize constant term

    for token in tokens
        if verbose println("Parsing token: $token") end
        token = strip(token)
        if isempty(token)
            if verbose println("Skipping empty token") end
            continue
        end

        # Determine the sign
        sign = 1.0
        if startswith(token, "-")
            sign = -1.0
            token = strip(token[2:end])
        elseif startswith(token, "+")
            token = strip(token[2:end])
        end

        if isempty(token)
            if verbose println("Skipping empty token after stripping sign") end
            continue
        end

        # Attempt to match variable terms
        m_var = match(r"^(\d*\.?\d*)\s*([A-Za-z][A-Za-z0-9_]*)$", token)
        if m_var !== nothing
            coeff_str, var = m_var.captures[1], m_var.captures[2]
            coeff = coeff_str == "" ? 1.0 : parse(Float64, coeff_str)
            coeff *= sign
            if verbose println("Parsed coefficient: $coeff for variable: $var") end
            coeff_dict[var] = get(coeff_dict, var, 0.0) + coeff
            push!(vars_set, var)
        else
            # Attempt to match constant terms
            m_const = match(r"^([+-]?\d+\.?\d*)$", token)
            if m_const !== nothing
                const_val = parse(Float64, m_const.captures[1])
                constant_term += sign * const_val
                if verbose println("Parsed constant term: $const_val") end
            else
                error("Failed to parse term: $token")
            end
        end
    end

    return constant_term
end

# Helper function to parse bounds
function parse_bounds(line::AbstractString, l_bounds::Dict{String, Float64}, u_bounds::Dict{String, Float64}, vars_set::Set{String}; verbose::Bool=false)
    tokens = split(line)
    if length(tokens) == 5 && (tokens[2] == "<=" || tokens[2] == ">=") && tokens[4] == tokens[2]
        # Format: lower <= variable <= upper
        lower = parse(Float64, tokens[1])
        var = tokens[3]
        upper = parse(Float64, tokens[5])
        l_bounds[var] = lower
        u_bounds[var] = upper
        push!(vars_set, var)
        if verbose println("Parsed bounds for variable $var: $lower <= $var <= $upper") end
    elseif length(tokens) == 3 && (tokens[2] == "<=" || tokens[2] == ">=")
        # Determine if tokens[1] is a variable or number
        is_var1 = occursin(r"^[A-Za-z_]", tokens[1])
        is_var3 = occursin(r"^[A-Za-z_]", tokens[3])
        if is_var1 && !is_var3
            # Format: variable <= upper OR variable >= lower
            var = tokens[1]
            bound = parse(Float64, tokens[3])
            if tokens[2] == "<="
                u_bounds[var] = bound
                if verbose println("Parsed upper bound for variable $var: $var <= $bound") end
            else  # ">="
                l_bounds[var] = bound
                if verbose println("Parsed lower bound for variable $var: $var >= $bound") end
            end
            push!(vars_set, var)
        elseif !is_var1 && is_var3
            # Format: lower <= variable OR upper >= variable
            bound = parse(Float64, tokens[1])
            var = tokens[3]
            if tokens[2] == "<="
                l_bounds[var] = bound
                if verbose println("Parsed lower bound for variable $var: $bound <= $var") end
            else  # ">="
                u_bounds[var] = bound
                if verbose println("Parsed upper bound for variable $var: $bound >= $var") end
            end
            push!(vars_set, var)
        else
            error("Unrecognized bounds format: $line")
        end
    else
        error("Unsupported bounds format: $line")
    end
end
```

</details>

In [ ]:
read_lp_test("../check/problems/lp_files/ex_9-7.lp", verbose=true)

In [ ]:
read_lp_test("../check/problems/lp_files/juLinear_ex1.lp", verbose=true)

In [ ]:
lp = read_lp("../check/problems/lp_files/problem.lp")

# Accessing different components
println("Objective is to minimize: ", lp.is_minimize)
println("Objective coefficients: ", lp.c)
println("Constraint matrix A: ", lp.A)
println("Right-hand side vector b: ", lp.b)
println("Constraint types: ", lp.constraint_types)
println("Lower bounds: ", lp.l)
println("Upper bounds: ", lp.u)
println("Variables: ", lp.vars)
println("Variable types: ", lp.variable_types)

## Checking against Highs ones

In [ ]:
mps_folder_path = "../check/problems/ls_files/"

In [ ]:
println(read_file_to_string("../check/problems/lp_files/test.lp"))